In [1]:
import csv
import datetime as datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import random
import os
from tqdm.notebook import trange, tqdm

In [2]:
def get_url(startPage):
    """Generate URL from position and Location
    :parameter: location -> Country"""
    template = 'https://empregos.org/jobfind.php?f={}&action=search&auth_sess=b32b5c79fb901f90536261a2e302a588&ref=558a97e97b671d3dca7fc7b67&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++'
    url = template.format(startPage)
    return url

In [3]:
url = get_url(5000)
url

'https://empregos.org/jobfind.php?f=5000&action=search&auth_sess=b32b5c79fb901f90536261a2e302a588&ref=558a97e97b671d3dca7fc7b67&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++'

# Extract Raw Html

In [4]:
response = requests.get(url)
response

<Response [200]>

In [6]:
soup = BeautifulSoup(response.text, 'html.parser')
entries = soup.find_all('td')
len(entries)

64

In [29]:
len(entries[12:-2])

50

In [30]:
cards = entries[12:-2].copy()
cards

[<td>
 <div class="rownumber"><strong>5001.</strong>
 <span class="bkm"><input class="radio" name="job_id[]" type="checkbox" value="2775447"/> </span>
 </div>
 <div class="results phpjob_listbgcolor21">
 <h2><a href="https://empregos.org/view.php?job_id=2775447&amp;type=search&amp;auth_sess=e261d448fae3a3354619fe256e5e7e88&amp;ref=1eb1266099101f7ad0ec7796e" title="Anúncio de emprego: Pizzaiolo M/F para Viseu em Pizzaria conceituada.">Anúncio de emprego: Pizzaiolo M/F para Viseu em Pizzaria conceituada.</a></h2>
 <p class="desc"><a class="hilink" href="https://empregos.org/view.php?company_id=59997&amp;type=search&amp;auth_sess=e261d448fae3a3354619fe256e5e7e88&amp;ref=1eb1266099101f7ad0ec7796e" title="GWF">GWF</a> - Viseu - Viseu<br>
 		Descrição: <h4>Pizzaiolo M/F para Viseu em Pizzaria conceituada.</h4>   <b>»</b>  GWF                                  <br> &amp;nbsp... <a class="hilink" href="https://empregos.org/view.php?job_id=2775447&amp;type=search&amp;auth_sess=e261d448fae3a33546

# Prototype the model with a single record

In [31]:
card = cards[0]
card

<td>
<div class="rownumber"><strong>5001.</strong>
<span class="bkm"><input class="radio" name="job_id[]" type="checkbox" value="2775447"/> </span>
</div>
<div class="results phpjob_listbgcolor21">
<h2><a href="https://empregos.org/view.php?job_id=2775447&amp;type=search&amp;auth_sess=e261d448fae3a3354619fe256e5e7e88&amp;ref=1eb1266099101f7ad0ec7796e" title="Anúncio de emprego: Pizzaiolo M/F para Viseu em Pizzaria conceituada.">Anúncio de emprego: Pizzaiolo M/F para Viseu em Pizzaria conceituada.</a></h2>
<p class="desc"><a class="hilink" href="https://empregos.org/view.php?company_id=59997&amp;type=search&amp;auth_sess=e261d448fae3a3354619fe256e5e7e88&amp;ref=1eb1266099101f7ad0ec7796e" title="GWF">GWF</a> - Viseu - Viseu<br>
		Descrição: <h4>Pizzaiolo M/F para Viseu em Pizzaria conceituada.</h4>   <b>»</b>  GWF                                  <br> &amp;nbsp... <a class="hilink" href="https://empregos.org/view.php?job_id=2775447&amp;type=search&amp;auth_sess=e261d448fae3a3354619fe256e

In [34]:
jobUrl = card.a['href']
jobUrl

'https://empregos.org/view.php?job_id=2775447&type=search&auth_sess=e261d448fae3a3354619fe256e5e7e88&ref=1eb1266099101f7ad0ec7796e'

In [35]:
jobTitle = card.a['title']
jobTitle

'Anúncio de emprego: Pizzaiolo M/F para Viseu em Pizzaria conceituada.'

In [36]:
shortSummary = card.p.text.strip().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
shortSummary

'GWF - Viseu - Viseu   Descrição: Pizzaiolo M/F para Viseu em Pizzaria conceituada.   »  GWF                                   &nbsp... Mais detalhes »'

# Get Full Job Description and Remaining Details

In [37]:
response = requests.get(jobUrl)
soup = BeautifulSoup(response.text, 'html.parser')
ref = soup.find('div', {'id':'idviewjob'})

In [278]:
jobTitle = ref.h3.string
jobTitle

'Vem ser mais connosco! Muda a tua vida!'

In [38]:
jobDescription = ref.p.text.strip()
jobDescription

'Pizzaiolo M/F para Viseu em Pizzaria conceituada.\xa0\xa0\xa0»  GWF                                  \xa0\xa0\xa0»  21-4-2021                          \xa0\xa0\xa0» Ref: 7369506                                                                  \xa0\xa0\xa0»   Viseu                                  \xa0\xa0\xa0»  Restauração / Bares / Pastelarias                                                        Detalhe da Oferta:                      \x95   Competências o   Experiência em funções similares; o   Domínio da língua Portuguesa; o   Conhecimentos de HACCP; o   Flexibilidade de horário; o   Espírito de equipa; o   Capacidade de aprendizagem. \x95   Responsabilidades o   Abertura de  massa, os recheios, o tempero e os aperitivos que acompanham uma pizza; o   Garantir a qualidade e o sabor do alimento, bem como o aspecto; o   Confecionar pizzas em forno a lenha; o   Limpar e higienizar toda a zona de trabalho. \x95   Benefícios Contrato de trabalho 9 meses; Formação inicial; Remuneração c

In [283]:
jobCategory = ref.table.find_all('td', 'col2')[0].text.replace('\xa0', ' ').split('[',1)[0][:-1]
languageReq = ref.table.find_all('td', 'col2')[1].text.strip()
jobType = ref.table.find_all('td', 'col2')[2].text.strip()
salary = ref.table.find_all('td', 'col2')[3].text.strip()
academicReq = ref.table.find_all('td', 'col2')[4].text.strip()
experienceYears = ref.table.find_all('td', 'col2')[5].text.strip()
jobLocation = ref.table.find_all('td', 'col2')[6].text.strip()
jobAdress = ref.find_all('td', 'view')[0].text.strip()
jobPostalCode = ref.find_all('td', 'view')[1].text.strip()
companyType = ref.find_all('tr')[9].contents[3].text.strip()
postDate = ref.table.find_all('td', 'col2')[7].text.strip()

if 'Empresa:' in [i.text.strip() for i in ref.find_all('strong')]:
    company = ref.find_all('a')[1]
else:
    company = ''
    
contactName = ref.table.find_all('td', 'col2')[8].text.strip()
contactEmail = ref.table.find_all('td', 'col2')[9].text.strip()

In [316]:
ref.find_all('strong')[0].parent.parent.find_all('td')[1].text.strip().replace('\xa0', '').split('[',1)[0]

'Comercial'

In [ ]:
for i in ref.find_all('strong'):
    id = i.text
    if id == 'Categoria de Emprego:':
        try:
            jobCategory = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobCategory = ''
    elif id == 'Requisitos linguísticos:':
        try:
            languageReq = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            languageReq = ''
    elif id == 'Tipo de emprego:':
        try:    
            jobType = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobType = ''
    elif id == 'Remuneração:':
        try:
            salary = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            salary = ''
    elif id == 'Grau académico:':
        try:    
            academicReq = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            academicReq = ''
    elif id == 'Anos de experiência:':
        try:
            experienceYears = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            experienceYears = ''
    elif id == 'Localização do emprego:':
        try:
            jobLocation = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobLocation = ''
    elif id == 'Endereço:':
        try:
            jobAdress = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobAdress = ''
    elif id == 'Código postal':
        try:
            jobPostalCode = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobPostalCode = ''
    elif id == 'Benefícios e outras informações:':
        try:
            jobBenefits = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobBenefits = ''
    elif id == 'Tipo de empresa':
        try:    
            companyType = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            companyType = ''
    elif id == 'Data de publicação:':
        try:
            postDate = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            postDate = ''
    elif id == 'Empresa:':
        try:
            company = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            company = ''
    elif id == 'Nome do contacto:':
        try:
            contactName = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            contactName = ''
    elif id == 'Email de contacto:':
        try:
            contactEmail = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            contactEmail = ''
    elif id == 'Endereço:':
        try:
            jobAdress = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            jobAdress = ''
    elif id == 'Telefone:':
        try:
            contactPhone = i.parent.parent.find_all('td')[1].text.strip()
        except AttributeError:
            contactPhone = ''

    
    

In [166]:
jobCategory = ref.table.find_all('td', 'col2')[0].text.replace('\xa0', ' ').split('[',1)[0][:-1]
jobCategory

'Comercial'

In [168]:
languageReq = ref.table.find_all('td', 'col2')[1].text.strip()
languageReq

''

In [173]:
jobType = ref.table.find_all('td', 'col2')[2].text.strip()
jobType

'Full Time, Part Time, Estudante, Temporário'

In [174]:
salary = ref.table.find_all('td', 'col2')[3].text.strip()
salary

'1250 EUR'

In [175]:
academicReq = ref.table.find_all('td', 'col2')[4].text.strip()
academicReq 

'não especificado'

In [180]:
experienceYears = ref.table.find_all('td', 'col2')[5].text.strip()
experienceYears

'Não especificado'

In [181]:
jobLocation = ref.table.find_all('td', 'col2')[6].text.strip()
jobLocation

'Setúbal'

In [186]:
postDate = ref.table.find_all('td', 'col2')[7].text.strip()
postDate

'2021-09-17 / Viewed 13 times'

In [205]:
contactName = ref.table.find_all('td', 'col2')[8].text.strip()
contactName

'Daniel Simões'

In [208]:
contactEmail = ref.table.find_all('td', 'col2')[9].text.strip()
contactEmail

'daniel.simoes@maisconsultores.pt'

In [212]:
jobAdress = ref.find_all('td', 'view')[0].text.strip()
jobAdress

'Rua Casal do Marco n.º 65A'

In [214]:
jobPostalCode = ref.find_all('td', 'view')[1].text.strip()
jobPostalCode

'2840-731'

In [246]:
companyType = ref.find_all('tr')[9].contents[3].text.strip()
companyType

'Recrutador'

In [258]:
if 'Empresa:' in [i.text.strip() for i in ref.find_all('strong')]:
    company = ref.find_all('a')[1]
else:
    company = ''

company

''

In [317]:
def get_jobDescription(jobUrl):
    response = requests.get(jobUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    ref = soup.find('div', {'id':'idviewjob'})

    # Full job description
    jobDescription = ref.p.text.strip()

    for i in ref.find_all('strong'):
        id = i.text
        if id == 'Categoria de Emprego:':
            try:
                jobCategory = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobCategory = ''
        elif id == 'Requisitos linguísticos:':
            try:
                languageReq = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                languageReq = ''
        elif id == 'Tipo de emprego:':
            try:    
                jobType = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobType = ''
        elif id == 'Remuneração:':
            try:
                salary = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                salary = ''
        elif id == 'Grau académico:':
            try:    
                academicReq = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                academicReq = ''
        elif id == 'Anos de experiência:':
            try:
                experienceYears = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                experienceYears = ''
        elif id == 'Localização do emprego:':
            try:
                jobLocation = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobLocation = ''
        elif id == 'Endereço:':
            try:
                jobAdress = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobAdress = ''
        elif id == 'Código postal':
            try:
                jobPostalCode = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobPostalCode = ''
        elif id == 'Benefícios e outras informações:':
            try:
                jobBenefits = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobBenefits = ''
        elif id == 'Tipo de empresa':
            try:    
                companyType = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                companyType = ''
        elif id == 'Data de publicação:':
            try:
                postDate = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                postDate = ''
        elif id == 'Empresa:':
            try:
                company = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                company = ''
        elif id == 'Nome do contacto:':
            try:
                contactName = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactName = ''
        elif id == 'Email de contacto:':
            try:
                contactEmail = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactEmail = ''
        elif id == 'Endereço:':
            try:
                jobAdress = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobAdress = ''
        elif id == 'Telefone:':
            try:
                contactPhone = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactPhone = ''
        else:
            try:
                other = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                other = ''


    dataToAdd = (company, companyType, jobCategory, jobType, salary, jobDescription,   
                 jobLocation, jobAdress, jobPostalCode, languageReq, academicReq, 
                 experienceYears, jobBenefits, postDate, contactName, contactEmail, contactPhone, other)
    return dataToAdd

# Generalize the Model with a function

In [294]:
def get_record(card):
    """Extract Job dat from a single record"""
    # Job Url
    jobUrl = card.a['href']

    # Job Title
    jobTitle = card.a['title']
    
    # Short Summary
    shortSummary = card.p.text.strip().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')

    # Extract Date
    today = datetime.date.today().strftime('%Y-%m-%d')

    record = (jobUrl, jobTitle, shortSummary, today)
    return record

# Iterating By Page

In [26]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [266]:
soup.find('div', 'pgright').find_all('a')[-1].get('href')

'https://empregos.org/jobfind.php?f=50&action=search&auth_sess=783c86b44850879600652e80fb445abf&ref=3d9dcc1e0d8c8a56602367094&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++'

In [ ]:
while True:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    for card in cards:
        record = get_record()
        records.append(record)

    try:
        url = soup.find('div', 'pgright').find_all('a')[-1].get('href')
    except AttributeError:
        break

# Putting it all together

In [40]:
import csv
import datetime as datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import random
import os


def get_url(startPage):
    """Generate URL from position and Location
    :parameter: location -> Country"""
    template = 'https://empregos.org/jobfind.php?f={}&action=search&auth_sess=b32b5c79fb901f90536261a2e302a588&ref=558a97e97b671d3dca7fc7b67&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++'
    url = template.format(startPage)
    return url

def get_jobDescription(jobUrl):

    response = requests.get(jobUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        ref = soup.find('div', {'id':'idviewjob'})
    except AttributeError:
        ref = ''

    # If there is no active job description return all blanks - to be dropped in Data Cleaning
    if ref == None or '':
        company, companyType, jobCategory, jobType, salary, jobDescription = '', '', '', '', '', ''
        jobLocation, jobAdress, jobPostalCode, languageReq, academicReq = '', '', '', '', ''
        experienceYears, jobBenefits, postDate, contactName, contactEmail, contactPhone, other = '', '', '', '', '', '', ''

        dataToAdd = (company, companyType, jobCategory, jobType, salary, jobDescription,   
                jobLocation, jobAdress, jobPostalCode, languageReq, academicReq, 
                experienceYears, jobBenefits, postDate, contactName, contactEmail, contactPhone, other)

        return dataToAdd

    # Full job description
    try:
        jobDescription = ref.p.text.strip()
    except AttributeError:
        jobDescription = ''
        
    # initiate all vars
    company, companyType, jobCategory, jobType, salary = '', '', '', '', ''
    jobLocation, jobAdress, jobPostalCode, languageReq, academicReq = '', '', '', '', ''
    experienceYears, jobBenefits, postDate, contactName, contactEmail, contactPhone, other = '', '', '', '', '', '', ''
    

    for i in ref.find_all('strong'):
        id = i.text
        if id == 'Categoria de emprego:':
            try:
                jobCategory = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobCategory = ''
        elif id == 'Requisitos linguísticos:':
            try:
                languageReq = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                languageReq = ''
        elif id == 'Tipo de emprego:':
            try:    
                jobType = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobType = ''
        elif id == 'Remuneração:':
            try:
                salary = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                salary = ''
        elif id == 'Grau académico:':
            try:    
                academicReq = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                academicReq = ''
        elif id == 'Anos de experiência:':
            try:
                experienceYears = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                experienceYears = ''
        elif id == 'Localização do emprego:':
            try:
                jobLocation = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobLocation = ''
        elif id == 'Endereço:':
            try:
                jobAdress = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobAdress = ''
        elif id == 'Código postal':
            try:
                jobPostalCode = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobPostalCode = ''
        elif id == 'Benefícios e outras informações:':
            try:
                jobBenefits = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobBenefits = ''
        elif id == 'Tipo de empresa':
            try:    
                companyType = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                companyType = ''
        elif id == 'Data de publicação:':
            try:
                postDate = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                postDate = ''
        elif id == 'Empresa:':
            try:
                company = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                company = ''
        elif id == 'Nome do contacto:':
            try:
                contactName = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactName = ''
        elif id == 'Email de contacto:':
            try:
                contactEmail = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactEmail = ''
        elif id == 'Endereço:':
            try:
                jobAdress = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                jobAdress = ''
        elif id == 'Telefone:':
            try:
                contactPhone = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                contactPhone = ''
        else:
            try:
                other = i.parent.parent.find_all('td')[1].text.strip()
            except AttributeError:
                other = ''

    dataToAdd = (company, companyType, jobCategory, jobType, salary, jobDescription,   
                jobLocation, jobAdress, jobPostalCode, languageReq, academicReq, 
                experienceYears, jobBenefits, postDate, contactName, contactEmail, contactPhone, other)

    return dataToAdd
    


def get_record(card):
    """Extract Job dat from a single record"""
    # Job Url
    jobUrl = card.a['href']

    # Job Title
    jobTitle = card.a['title']
    
    # Short Summary
    shortSummary = card.p.text.strip().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')

    # Extract Date
    extractDate = datetime.date.today().strftime('%Y-%m-%d')

    dataToAdd = get_jobDescription(jobUrl)

    record = (jobTitle, shortSummary, jobUrl, extractDate) + dataToAdd

    return record

def main(startPage):
    """Run the main program routine"""
    records = []
    url = get_url(startPage)

    # Extract Job Data   
    iter = 0 
    # Break out of loop if waiting time > 3 min
    timeout = time.time() + 60*4
    while True:
        # Random Sleep Time 0 to 1 seconds
        time.sleep(random.random())
        iter += 1

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        entries = soup.find_all('td')
        cards = entries[12:-2].copy()

        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = soup.find('div', 'pgright').find_all('a')[-1].get('href')
            print(f'Iteration number {iter}')
            if iter == 50 or time.time() > timeout:
                print(url)
                break
        except AttributeError:
            print(url)
            print(f'Number of Iterations was {iter}')
            break

    # Save Job Data
    if os.path.exists('empregoOrg_jobs.csv') == False:
        with open('empregoOrg_jobs.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['jobTitle', 'shortSummary', 'jobUrl', 'extractDate', 
                'company', 'companyType', 'jobCategory', 'jobType', 'salary', 'jobDescription',   
                'jobLocation', 'jobAdress', 'jobPostalCode', 'languageReq', 'academicReq', 
                'experienceYears', 'jobBenefits', 'postDate', 'contactName', 'contactEmail', 'contactPhone', 'other'])
            writer.writerows(records)
        f.close()
    else:
        with open('empregoOrg_jobs.csv', 'a+', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerows(records)
        f.close()
    
    print(f'Scrapped {len(records)} job posting')

In [42]:
main(startPage=0)

Iteration number 1
Iteration number 2
Iteration number 3
Iteration number 4
Iteration number 5
Iteration number 6
Iteration number 7
Iteration number 8
Iteration number 9
Iteration number 10
Iteration number 11
Iteration number 12
Iteration number 13
Iteration number 14
Iteration number 15
Iteration number 16
Iteration number 17
Iteration number 18
Iteration number 19
Iteration number 20
https://empregos.org/jobfind.php?f=1000&action=search&auth_sess=68113a7c8081c425b16034a718557407&ref=ea2fd45810ecf508717f2411b&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++
Scrapped 1000 job posting


In [59]:
for i in tqdm(range(5)):
    try:
        start = len(pd.read_csv('empregoOrg_jobs.csv'))
        print(f'Start Page -> {start}')
        main(startPage=start)
        print(f'Total Jobs scraped {start}')
    except:
        break

  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\gilnr\Anaconda3\envs\mlcourse\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,12,13,16,18,19,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Start Page -> 65800
Iteration number 1
Iteration number 2
Iteration number 3
Iteration number 4
Iteration number 5
Iteration number 6
Iteration number 7
Iteration number 8
Iteration number 9
Iteration number 10
Iteration number 11
https://empregos.org/jobfind.php?f=66350&action=search&auth_sess=ae35bcf63d88774e5c3e59c110dac496&ref=b87a78fd501513b6a798f790a&bx_jtitle=&rdjt=2&jids[]=00&lids[]=000&bx_prv[]=&bx_plng[]=0&bx_kwd=&rdKeyw=2&bx_minsalary=&bx_maxsalary=&bx_lngids[]=-&rdLang=2&tids[]=0&posted=0&o=1&o_show=4&cmdSearch=++Pesquisar++


In [60]:
print(f'Number of scraped Job Ads -> {len(pd.read_csv("empregoOrg_jobs.csv"))}')
pd.read_csv('empregoOrg_jobs.csv')

Number of scraped Job Ads -> 65800


,jobTitle,shortSummary,jobUrl,extractDate,company,companyType,jobCategory,jobType,salary,jobDescription,...,jobPostalCode,languageReq,academicReq,experienceYears,jobBenefits,postDate,contactName,contactEmail,contactPhone,other
0,Gestor Comercial,century21 - Lisboa - Lisboa - Lisboa Descriç...,https://empregos.org/view.php?job_id=2796393&t...,2021-09-18,century21,Empregador,Comercial [ Ver todos os empregos de Comercial ],Full Time,15.000 a 25.000 EUR,View century21 profile and job listings,...,1500-506,NaN,não especificado,Não especificado,Formação de iniciação gratuita \r\nPlano de ...,2021-09-18 / Viewed 390 times,CENTURY 21 Colombo,rh.colombo@century21.pt,217106180,Envie-nos o seu CV atualizado para rh.colombo@...
1,Consultor Negócios - Porto,Maxgroup - Porto Descrição: Está à procura d...,https://empregos.org/view.php?job_id=2796392&t...,2021-09-18,Maxgroup,Empregador,Imobiliário [ Ver todos os empregos de Imobili...,"Full Time, Permanente",25.000 Ano EUR,Está à procura de um Novo Desafio?\r\nValoriza...,...,4000-238 Porto,NaN,não especificado,Não especificado,NaN,2021-09-18 / Viewed 216 times,Daniel Renato,drenato@remax.pt,963014877,talent_27_08_2021
2,Gestor de Cliente Imobiliário ( m/f ),Pole Position Lda - Lisboa - Lisboa Descriçã...,https://empregos.org/view.php?job_id=2796391&t...,2021-09-18,Pole Position Lda,Empregador,Comercial [ Ver todos os empregos de Comercial ],Full Time,Não especificado,S E J A C E N T U R Y 2 1. S E J A P O L E P O...,...,2720-306,NaN,não especificado,Não especificado,NaN,2021-09-18 / Viewed 469 times,Ana Simal,anaisabel.simal@century21.pt,215 806 787,NaN
3,Part-Time - Possibilidade de teletrabalho - Mí...,- Setúbal Descrição: Gostava de poder mudar ...,https://empregos.org/view.php?job_id=2796390&t...,2021-09-18,NaN,Recrutador,Comercial [ Ver todos os empregos de Comercial ],"Full Time, Part Time, Estudante, Temporário",1250 EUR,Gostava de poder mudar a sua vida? De poder te...,...,2840-731,NaN,não especificado,Não especificado,NaN,2021-09-18 / Viewed 485 times,Daniel Simões,daniel.simoes@maisconsultores.pt,NaN,NaN
4,Comercial C/ ou S/ Experiência Imobiliário 50%...,- Setúbal Descrição: Gostava de poder mudar ...,https://empregos.org/view.php?job_id=2796389&t...,2021-09-18,NaN,Recrutador,Comercial [ Ver todos os empregos de Comercial ],"Full Time, Part Time, Estudante, Temporário",1250 EUR,Gostava de poder mudar a sua vida? De poder te...,...,2840-731,NaN,não especificado,Não especificado,NaN,2021-09-18 / Viewed 222 times,Daniel Simões,daniel.simoes@maisconsultores.pt,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65795,Oferta: Precisa-se Empregado de Mesa (part tim...,"Forneria São Dinis - Restauração, Lda - Outro ...",https://empregos.org/view.php?job_id=2790489&t...,2021-09-20,"Forneria São Dinis - Restauração, Lda",Empregador,Outras [ Ver todos os empregos de Outras ],NaN,Não especificado,Precisa-se Empregado de Mesa (part time) para ...,...,NaN,NaN,não especificado,Não especificado,NaN,2021-09-03 / Viewed 165 times,NaN,NaN,NaN,NaN
65796,Oferta de emprego: Precisa-se Copeiro(a) para ...,"Forneria São Dinis - Restauração, Lda - Outro ...",https://empregos.org/view.php?job_id=2790488&t...,2021-09-20,"Forneria São Dinis - Restauração, Lda",Empregador,Outras [ Ver todos os empregos de Outras ],NaN,Não especificado,Precisa-se Copeiro(a) para Restaurante de Reno...,...,NaN,NaN,não especificado,Não especificado,NaN,2021-09-03 / Viewed 139 times,NaN,NaN,NaN,NaN
65797,Oferta: Precisa-se Ajudante de Cozinha para Re...,"Forneria São Dinis - Restauração, Lda - Outro ...",https://empregos.org/view.php?job_id=2790487&t...,2021-09-20,"Forneria São Dinis - Restauração, Lda",Empregador,Outras [ Ver todos os empregos de Outras ],NaN,Não especificado,Precisa-se Ajudante de Cozinha para Restaurant...,...,NaN,NaN,não especificado,Não especificado,NaN,2021-09-03 / Viewed 137 times,NaN,NaN,NaN,NaN
65798,Oferta: Operadores de valores (M/F) AÇORES Ful...,ESEGUR